In [1]:
import torch
import dgl
import time
import numpy as np
import gs
from gs.utils import SeedGenerator
import numba
from numba.core import types
from numba.typed import Dict
from dgl.utils import gather_pinned_tensor_rows
from ogb.nodeproppred import DglNodePropPredDataset
import tqdm
import os


# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


def load_ogb(name):
    data = DglNodePropPredDataset(name=name, root="../datasets")
    splitted_idx = data.get_idx_split()
    g, labels = data[0]
    g = g.long()
    feat = g.ndata['feat']
    labels = labels[:, 0]
    n_classes = len(torch.unique(
        labels[torch.logical_not(torch.isnan(labels))]))
    g.ndata.clear()
    g = dgl.remove_self_loop(g)
    g = dgl.add_self_loop(g)
    return g, feat, labels, n_classes, splitted_idx

/home/ubuntu/anaconda3/envs/dgl/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_ogb('ogbn-products')

In [3]:
dgl_graph = dataset[0]
g = dgl_graph.long()
probs = g.out_degrees().float()

In [6]:
g = g.to('cuda')

In [9]:
print(torch.cuda.memory_allocated() / (1024 * 1024 * 1024), 'GB')

3.778319835662842 GB


In [10]:
g.formats()

{'created': ['coo', 'csr'], 'not created': ['csc']}

In [11]:
nodes = g.nodes().cuda()
probs_cuda = probs.cuda()

In [12]:
print(torch.cuda.memory_allocated() / (1024 * 1024 * 1024), 'GB')

3.8056907653808594 GB


In [13]:
seed_nodes = nodes[:2048]

In [14]:
print(torch.cuda.memory_allocated() / (1024 * 1024 * 1024), 'GB')

3.8056907653808594 GB


In [20]:
torch.cuda.reset_peak_memory_stats()
subg = dgl.in_subgraph(g, seed_nodes)
print(torch.cuda.max_memory_allocated() / (1024 * 1024 * 1024), 'GB')

5.722006320953369 GB


In [17]:
g.formats()

{'created': ['coo', 'csr', 'csc'], 'not created': []}

In [16]:
subg.formats()

{'created': ['coo'], 'not created': ['csr', 'csc']}

In [18]:
print(torch.cuda.memory_allocated() / (1024 * 1024 * 1024), 'GB')

5.711845397949219 GB


In [18]:
edges = subg.edges()
nodes = torch.unique(edges[0])  # coo row

In [20]:
print(torch.cuda.memory_allocated() / (1024 * 1024 * 1024), 'GB')

1.2423677444458008 GB


In [23]:
torch.cuda.reset_peak_memory_stats()
num_pick = np.min([nodes.shape[0], 2000])
node_probs = probs[nodes]
idx = torch.multinomial(node_probs, num_pick, replacement=False)  # gpu
selected = nodes[idx]  # gpu
print(torch.cuda.max_memory_allocated() / (1024 * 1024 * 1024), 'GB')

1.2423982620239258 GB


In [24]:
print(torch.cuda.memory_allocated() / (1024 * 1024 * 1024), 'GB')

1.2423830032348633 GB


In [26]:
torch.cuda.reset_peak_memory_stats()
out_subg = dgl.out_subgraph(subg, selected)
print(torch.cuda.max_memory_allocated() / (1024 * 1024 * 1024), 'GB')

1.2423830032348633 GB


In [27]:
subg.edges()[0].numel()

24150

In [28]:
out_subg.edges()[0].numel()

2007

In [29]:
print(torch.cuda.memory_allocated() / (1024 * 1024 * 1024), 'GB')

1.2423830032348633 GB


In [30]:
subg.formats()

{'created': ['coo', 'csr'], 'not created': ['csc']}

In [31]:
subg.num_nodes()

111059956

In [32]:
g.formats()

{'created': ['coo', 'csr', 'csc'], 'not created': []}

In [33]:
torch.cuda.reset_peak_memory_stats()
temp = torch.ones(111059956, dtype=torch.long, device='cuda')
print(torch.cuda.max_memory_allocated() / (1024 * 1024 * 1024), 'GB')

2.0705080032348633 GB


In [34]:
print(torch.cuda.memory_allocated() / (1024 * 1024 * 1024), 'GB')

2.0705080032348633 GB


In [36]:
torch.cuda.reset_peak_memory_stats()
csr_subg = subg.formats('csr')
print(torch.cuda.max_memory_allocated() / (1024 * 1024 * 1024), 'GB')

2.0705080032348633 GB


In [38]:
csr_subg.metagraph()

In [41]:
csr_subg.metagraph().adj

MultiAdjacencyView({'_N': {'_N': {'_E': {}}}})

In [42]:
csr_subg.num_nodes()

111059956

In [43]:
print(torch.cuda.memory_allocated() / (1024 * 1024 * 1024), 'GB')

2.0705080032348633 GB


In [45]:
torch.cuda.reset_peak_memory_stats()
subg.create_formats_()
print(torch.cuda.max_memory_allocated() / (1024 * 1024 * 1024), 'GB')

2.0705080032348633 GB


In [46]:
subg.formats()

{'created': ['coo', 'csr', 'csc'], 'not created': []}

In [47]:
csr_subg.idtype

torch.int64

In [48]:
print(torch.cuda.memory_allocated() / (1024 * 1024 * 1024), 'GB')

2.0705080032348633 GB


In [49]:
csr_subg

Graph(num_nodes=111059956, num_edges=24150,
      ndata_schemes={}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})

In [50]:
print(torch.cuda.memory_allocated() / (1024 * 1024 * 1024), 'GB')

2.0705080032348633 GB


In [54]:
torch.cuda.reset_peak_memory_stats()
csr_subg.metagraph()
print(torch.cuda.max_memory_allocated() / (1024 * 1024 * 1024), 'GB')

2.0705080032348633 GB
